In [1]:
import requests
from bs4 import BeautifulSoup
import time
from urllib import urlencode
import selenium.webdriver
import random
from pymongo import MongoClient
import json
import boto3
import pandas as pd

In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client.route_html_collection
route_html_collection = db.route_html_collection
raw_data = route_html_collection.find()
df_route_html = pd.DataFrame(list(raw_data))
df_route_html.head()

,30(1) Feet of Pleasure,_id
0,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",58c8735040b44102a84aec42
1,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",58c8736040b44102a84aec48
2,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",58c8736f40b44102a84aec4e
3,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",58c8737d40b44102a84aec54
4,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",58c8738b40b44102a84aec5a


In [6]:
client = MongoClient('mongodb://localhost:27017/')
db = client.user_html_collection
user_html_collection = db.user_html_collection
raw_data = user_html_collection.find()
df_user_html = pd.DataFrame(list(raw_data))
df_user_html.head()

,Abandoned User,Andrew Davidson,BenCooper,Benjamin Smith,BrianWilson,Chris Culver,Chris Walden,Curtveld,Dakota Bauman,Dan Sulpizi,...,Tom Bath,Travis A,_id,adam winslow,applewood,dondo,jubalee,laurichj,michael faith,slim
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,58c8735940b44102a84aec46,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,58c8736c40b44102a84aec4c,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,58c8737640b44102a84aec52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,58c8738640b44102a84aec58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",NaN,NaN,NaN,NaN,...,NaN,NaN,58c8739540b44102a84aec5e,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
from parse_clean_store import parse_route_page
from bs4 import BeautifulSoup


In [75]:
soup = BeautifulSoup(html, 'html.parser')
page_tag = soup.find('div', {'id':'rspCol800'})
# make route dict
route_dict = {}
route_dict['name'] = page_tag.find('span', {'itemprop':'itemreviewed'}).text.encode('utf-8')
route_dict['grade'] = page_tag.find('span', {'class':'rateYDS'}).text
route_stars_text = soup.find('span', {'id':'starSummaryText'}).text.split('Average: ')
route_dict['average_rating'] = route_stars_text[1][:3]

In [143]:
def parse_route_page(html):
    '''
    takes a soup to parse
    returns dict of route info
    '''
    soup = BeautifulSoup(html, 'html.parser')
    page_tag = soup.find('div', {'id':'rspCol800'})
    # make route dict
    route_dict = {}
    route_dict['name'] = page_tag.find('span', {'itemprop':'itemreviewed'}).text
    route_dict['grade'] = clean_grade(page_tag)
    route_dict['average_rating'] = clean_average_rating(soup)
    
    for i, td in enumerate(page_tag.find('table').findAll('td')): 
        if td.text.split(':')[0] == 'Type':
            route_type = clean_route_type(page_tag.find('table').findAll('td')[i+1])
            route_dict['route_type_list'] = route_type[0]
            route_dict['pitches'] = route_type[1]
            route_dict['height'] = route_type[2]
            route_dict['uiaa'] = route_type[3]
            route_dict['misc'] = route_type[4]
        elif td.text.split(':')[0] == 'Original':
            route_dict['original_grade'] = clean_original_grade(page_tag.find('table').findAll('td')[i+1])
        elif td.text.split(':')[0] == 'FA':
            fa = page_tag.find('table').findAll('td')[i+1].text
            route_dict['FA'] = fa
        elif td.text.split(':')[0] == 'Season':
            season = page_tag.find('table').findAll('td')[i+1].text
            route_dict['season'] = season.encode('utf-8')
        elif td.text.split(':')[0] == 'Page Views':
            page_view = page_tag.find('table').findAll('td')[i+1].text
            route_dict['page_views'] = int(page_view.encode('utf-8').replace(',',''))
        elif td.text.split(':')[0] == 'Submitted By':
            sb1, sb2 = clean_submitted_by(page_tag.find('table').findAll('td')[i+1])
            route_dict['submitted_by'] = sb1
            route_dict['submitted_on'] = sb2
            
    return route_dict


def clean_route_type(tag):
    route_type = tag.text
    route_type = route_type.encode('utf-8').split(', ')
    list_of_types = ['Trad', 'Sport', 'TR', 'Alpine', 
                    'Ice', 'Boulder', 'Aid', 'Mixed']
    # initialize variables
    route_type_list = []
    pitches = float('nan')
    height = float('nan')
    uiaa = float('nan')
    misc = float('nan')
    for item in route_type:
        if item in list_of_types:
            route_type_list.append(item)
        elif 'pitches' in item:
            pitches = item.replace(' pitches', '')
        elif 'pitch' in item:
            pitches = item.replace(' pitch','')
        elif "'" in item:
            height = float(item.replace("'",''))
        elif 'Grade' in item:
            uiaa = item
        else:
            misc = item

    return route_type_list, pitches, height, uiaa, misc


def clean_submitted_by(page_tag):
    sb = page_tag.text
    sb = sb.encode('utf-8').split(' on ')
    return sb[0], sb[1]

def clean_original_grade(page_tag):
    grade = page_tag.text
    grade = grade.encode('utf-8').split('\xa0')[1]
    grade = grade.split('French')[0]
    return grade

def clean_grade(page_tag):
    grade = page_tag.find('span', {'class':'rateYDS'}).text
    grade = grade.encode('utf-8').split('\xa0')[1]
    return grade

def clean_average_rating(soup):
    route_stars_text = soup.find('span', {'id':'starSummaryText'}).text.split('Average: ')
    average_rating = route_stars_text[1][:3]
    if 'OK' in average_rating:
        average_rating = 1.0
    elif 's' in average_rating:
        average_rating = float(average_rating[0])
    else:
        average_rating = float(average_rating)
    return average_rating

In [144]:
dict_list = []
for html in df_route_html['30(1) Feet of Pleasure']:
    dict_list.append(parse_route_page(html))

In [145]:
df = pd.DataFrame(dict_list)
df

,FA,average_rating,grade,height,misc,name,original_grade,page_views,pitches,route_type_list,season,submitted_by,submitted_on,uiaa
0,"Adam Winslow, Bill Coe, Kyle Silverman 3/20/09",1.0,5.6,41.0,NaN,30(1) Feet of Pleasure,5.6,588,1,[Trad],NaN,adam winslow,"Aug 18, 2010",NaN
1,"Adam Winslow, Ujahn Davisson, Bill Coe, Jim Op...",1.0,5.5,41.0,NaN,41 Feet of Pain,5.5,642,1,[Trad],NaN,adam winslow,"Aug 18, 2010",NaN
2,?,1.0,5.5,35.0,NaN,5.5 Crack,5.5,176,NaN,"[Trad, TR]",Afternoon shade,CHopwood,"Sep 2, 2013",NaN
3,,2.0,5.6,85.0,NaN,5.6 crack,5.6,1263,1,"[Trad, TR]",NaN,Tom Bath,"Dec 16, 2009",NaN
4,Unknown,1.4,5.6,30.0,NaN,5.6 Dihedral,5.6,678,1,"[Trad, TR]",NaN,chrishar Hargarten,"May 19, 2008",NaN
5,,2.0,5.7,40.0,NaN,5.7,5.7,326,NaN,[Sport],NaN,Keenan Waeschle,"Mar 31, 2010",NaN
6,,1.7,5.7,95.0,NaN,5.7 Face,5.7,454,1,"[Trad, TR]",NaN,Mark Goodro,"Jun 18, 2013",NaN
7,,2.0,5.5,95.0,NaN,A,5.5,18,NaN,[Sport],NaN,applewood,"May 28, 2013",NaN
8,,1.0,5.7,NaN,NaN,A Crack,5.7,51,NaN,[Trad],NaN,Blitzo,"Oct 18, 2006",NaN
9,10/13,3.0,5.3,75.0,NaN,A MIGHTY WIND,5.3,39,NaN,[Sport],NaN,applewood,"Oct 27, 2013",NaN


In [66]:
df['grade'].unique()[0].encode('utf-8').split('\xa0')[1]

'5.6'

In [115]:
df['season'].unique()[0]

nan

In [116]:
d = float('nan')
d

nan

In [146]:
client = MongoClient('mongodb://localhost:27017/')
db = client.route_html_collection
route_html_collection = db.route_html_collection
raw_data = route_html_collection.find()
df_route_html2 = pd.DataFrame(list(raw_data))
df_route_html2.head()

,_id,html,route
0,58c87f8840b44102a84aedf9,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",30(1) Feet of Pleasure
1,58c87f9740b44102a84aedff,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",41 Feet of Pain
2,58c87fa640b44102a84aee05,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.5 Crack
3,58c87fb140b44102a84aee0b,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 crack
4,58c87fbc40b44102a84aee11,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 Dihedral


In [152]:
dict_list = []
for html in df_route_html2['html']:
    dict_list.append(parse_route_page(html))
df = pd.DataFrame(dict_list)
df.head()

,FA,average_rating,grade,height,misc,name,original_grade,page_views,pitches,route_type_list,season,submitted_by,submitted_on,uiaa
0,"Adam Winslow, Bill Coe, Kyle Silverman 3/20/09",1.0,5.6,41.0,NaN,30(1) Feet of Pleasure,5.6,588,1,[Trad],NaN,adam winslow,"Aug 18, 2010",NaN
1,"Adam Winslow, Ujahn Davisson, Bill Coe, Jim Op...",1.0,5.5,41.0,NaN,41 Feet of Pain,5.5,642,1,[Trad],NaN,adam winslow,"Aug 18, 2010",NaN
2,?,1.0,5.5,35.0,NaN,5.5 Crack,5.5,176,NaN,"[Trad, TR]",Afternoon shade,CHopwood,"Sep 2, 2013",NaN
3,,2.0,5.6,85.0,NaN,5.6 crack,5.6,1263,1,"[Trad, TR]",NaN,Tom Bath,"Dec 16, 2009",NaN
4,Unknown,1.4,5.6,30.0,NaN,5.6 Dihedral,5.6,678,1,"[Trad, TR]",NaN,chrishar Hargarten,"May 19, 2008",NaN


In [147]:
client = MongoClient('mongodb://localhost:27017/')
db = client.user_html_collection
user_html_collection = db.user_html_collection
raw_data = user_html_collection.find()
df_user_html2 = pd.DataFrame(list(raw_data))
df_user_html2.head()

,_id,html,username
0,58c87f9340b44102a84aedfd,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",adam winslow
1,58c87fa440b44102a84aee03,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",adam winslow
2,58c87fae40b44102a84aee09,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Oscar Edge
3,58c87fb640b44102a84aee0f,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Nick Drake
4,58c87fc340b44102a84aee15,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Chris Culver


In [149]:
client = MongoClient('mongodb://localhost:27017/')
db = client.ratings_collection
ratings_collection = db.ratings_collection
raw_data = ratings_collection.find()
df_ratings2 = pd.DataFrame(list(raw_data))
df_ratings2.head()

,_id,rating,route,username
0,58c87f8c40b44102a84aedfb,[1],30(1) Feet of Pleasure,[adam winslow ]
1,58c87f9d40b44102a84aee01,[1],41 Feet of Pain,[adam winslow ]
2,58c87fac40b44102a84aee07,"[1, 1, 1, 1]",5.5 Crack,"[CHopwood , Trevor. , Rickedge , Oscar Edge ]"
3,58c87fb440b44102a84aee0d,"[3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",5.6 crack,"[Bianca P , Sandeep Kumar , Mike Brady , Tom B..."
4,58c87fbf40b44102a84aee13,"[2, 2, 2, 2, 1, 1, 1, 1, 1]",5.6 Dihedral,"[Jason Killgore , sqwirll , Dave Leydet , fort..."
